<a href="https://colab.research.google.com/github/khanh22mse23057/python4engineer-assignments/blob/main/Assignments3_M5_Forecasting_with_LSTM_and_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tìm hiểu về LSTM and LightGBM

##  *Lời mở đầu*

Long short-term memory (LSTM) là một kiến ​​trúc artificial recurrent neural network (RNN) được sử dụng trong lĩnh vực Deep learning. 
Light gradient-boost machine(LightGBM), là một khuôn khổ tăng cường độ dốc phân tán miễn phí và mã nguồn mở dành cho máy học, ban đầu được phát triển bởi Microsoft. Nó dựa trên các thuật toán cây quyết định và được sử dụng để xếp hạng, phân loại và các tác vụ học máy khác


### **Cài đặt môi trường**
Để truy xuất được dữ liệu từ Kaggle cũng như chạy được các đoạn mã ví dụ cần phải cài đặt một số packages

In [ ]:
# Một số functions trong bài không work với latest version, nên cài đặt packages version chỉ định như sau
# !pip install opendatasets
# !pip install pandas==1.2.0
# !pip install pystan==3.0.0
!pip install pandas 

!pip install kaggle
!pip install unzip


### Import tệp dữ liệu training từ Kaggle
Để truy cập được tệp data từ Kaggle, cần phải tạo key và cấu hình môi trường

Ví dụ file secret key được tạo từ Kaggle và được lấy về từ github. Cách tạo key : https://prnt.sc/UCwb6zgoPzYT
* Lưu ý: Kaggle yêu cầu xác thực tài khoản bằng số điện thoại để cấp cấp quyền truy cập dữ liệu.

In [ ]:
import os
#Cấu hình secret key để xác thực truy cập đến Kaggle trên môi trường
os.environ['KAGGLE_USERNAME'] = 'khanhphamnguyenphu'
os.environ['KAGGLE_KEY'] = '67b96e181bd01730f8e8ec3cf2b685a7'

!if [ -d ~/.kaggle/ ]; then rm -Rf ~/.kaggle/; fi
!if [ -d ./kaggle/input/ ]; then rm -Rf ./kaggle/input/; fi

!kaggle competitions download -c m5-forecasting-accuracy -p ./kaggle/input/ # Download dữ liệu train từ Kaggle
!unzip  ./kaggle/input/*.zip -d ./kaggle/input/
!ls ./kaggle/input/ 


 68% 31.0M/45.8M [00:00<00:00, 157MB/s]
100% 45.8M/45.8M [00:00<00:00, 151MB/s]
Archive:  ./kaggle/input/m5-forecasting-accuracy.zip
  inflating: ./kaggle/input/calendar.csv  
  inflating: ./kaggle/input/sales_train_evaluation.csv  
  inflating: ./kaggle/input/sales_train_validation.csv  
  inflating: ./kaggle/input/sample_submission.csv  
  inflating: ./kaggle/input/sell_prices.csv  
calendar.csv		     sales_train_evaluation.csv  sample_submission.csv
m5-forecasting-accuracy.zip  sales_train_validation.csv  sell_prices.csv
